In [3]:
import numpy as np 
import cupy as cp
import tiffile as tiff

## Cylinder mask

In [4]:
def circle_mask(size, radius):
    mask = np.zeros([size, size, size], dtype = np.uint8)
    X, Y = np.ogrid[:size, :size]
    dist_from_center = np.sqrt((X - size // 2)**2 + (Y - size // 2)**2, dtype = np.float32)
    mask = dist_from_center <= radius
    return mask

## Sphere generator

In [5]:
def circle(size, center, radius):
    mask = np.zeros([size, size, size], dtype = np.uint8)
    X, Y, Z = np.ogrid[:size, :size, :size]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = np.float32)
    mask = dist_from_center<= radius
    return mask 

def circle_cuda(size, center, radius):
    mask = cp.zeros([size, size, size], dtype = cp.uint8)
    X, Y, Z = cp.ogrid[:size, :size, :size]
    dist_from_center = cp.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = cp.float32)
    mask = dist_from_center<= radius

## 3D spheres packing generation 

In [8]:
# number of iterations
N = 200

# box size
size = 512

# maximum value of sphere radius variation (R_min + rand(0, 1)*delta)
delta = 30 

# minimal sphere radius
r_min = 30

def circle(size, center, radius):
    mask = np.zeros([size, size, size], dtype = np.uint8)
    X, Y, Z = np.ogrid[:size, :size, :size]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = np.float32)
    mask = dist_from_center.astype(np.float32)<= radius
    return mask 

def circle_cuda(size, center, radius):
    mask = cp.zeros([size, size, size], dtype = cp.uint8)
    X, Y, Z = cp.ogrid[:size, :size, :size]
    dist_from_center = cp.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = cp.float32)
    mask = dist_from_center.astype(cp.float32)<= radius
    return mask

box = cp.zeros([size, size, size], dtype = cp.uint8)

x0 = cp.random.randint(0, size, 1)
y0 = cp.random.randint(0, size, 1)
z0 = cp.random.randint(0, size, 1)
r0 = r_min + cp.random.random() * delta

cent = [[x0, y0, z0]]
radius = [r0]

box += circle_cuda(size = size, center = [x0, y0, z0], radius = r0)

# iterator
i = 0

while (i < N):
    
    #print(i)
    
    xi = cp.random.randint(0, size, 1)
    yi = cp.random.randint(0, size, 1)
    zi = cp.random.randint(0, size, 1)
    
    ri = r_min + cp.random.random() * delta
    
    circle_i = circle_cuda(size = size, center = [xi, yi, zi], radius = ri)
    
    if (len((box + circle_i)[cp.where((box + circle_i) == 2)]) == 0):
        
        box += circle_i
        cent.append([xi, yi, zi])
        radius.append(ri)
        
    i+=1

## Move spheres

In [9]:
# number of iterations 
N = 50 

# maximal step size in pixels  
step = 5

# iterator
i = 0 

while (i < N):
    
    print(i)
    
    box = cp.zeros([size, size, size], dtype = cp.uint8)
    
    cent = cp.array(cent)
    
    x = cent[:, 0]
    y = cent[:, 1]
    z = cent[:, 2]

    idx = int(np.random.randint(0, len(cent), 1))
    
    dx = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    dy = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    dz = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    
    x[idx] = x[idx] + dx
    y[idx] = y[idx] + dy
    z[idx] = z[idx] + dz

    r = radius
    
    # save spheres radiuses
    np.save('move_3D/radius.npy', r)
    
    for j in range(len(cent)):
        circle_i = circle_cuda(size = size, center = [x[j], y[j], z[j]], radius = r[j])
        box += circle_i

    if (len(box[cp.where(box == 2)]) == 0): 
               
        cent[:, 0] = x[:]
        cent[:, 1] = y[:]
        cent[:, 2] = z[:]
        
        # save spheres packing 
        tiff.imwrite('move_3D/frame_' + str(i) + '.tiff', box.get() * 255)
        # save centers coordinates 
        np.save('move_3D/coord_' + str(i) + '.npy', cent.get())
        
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
